In [1]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

# This secure connect bundle is autogenerated when you donwload your SCB, 
# if yours is different update the file name below
cloud_config= {
  'secure_connect_bundle': 'secure-connect-test-db.zip'
}

# This token json file is autogenerated when you donwload your token, 
# if yours is different update the file name below
with open("test_db-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
if row:
    print(row[0])
else:
    print("An error occurred.")

4.0.0.6816


In [2]:
session.execute("insert into kk.products (id,name,category,price) values (6 ,'p6','c2',453.7)")

In [42]:
result=session.execute("select * from kk.products")

In [15]:
for r in result:
    print(r.id)

5
1
2
4
6
3


In [8]:
list(result)

[Row(id=5, category='c3', name='p5', price=5634.89990234375),
 Row(id=1, category='c1', name='p1', price=234.89999389648438),
 Row(id=2, category='c1', name='p2', price=2334.89990234375),
 Row(id=4, category='c1', name='p4', price=6634.89990234375),
 Row(id=6, category='c2', name='p6', price=453.70001220703125),
 Row(id=3, category='c2', name='p3', price=534.9000244140625)]

In [43]:
data_prod={'id':[],'name':[],'category':[],'price':[]}

In [44]:
for r in result:
    data_prod['id'].append(r.id)
    data_prod['name'].append(r.name)
    data_prod['category'].append(r.category)
    data_prod['price'].append(r.price)

In [45]:
data_prod

{'id': [5, 1, 2, 4, 3],
 'name': ['p5', 'p1', 'p2', 'p4', 'p3'],
 'category': ['c3', 'c1', 'c4', 'c1', 'c2'],
 'price': [5634.89990234375,
  234.89999389648438,
  2334.89990234375,
  6634.89990234375,
  534.9000244140625]}

In [47]:
import pandas as pd
df_prod=pd.DataFrame(data_prod)
df_prod

,id,name,category,price
0,5,p5,c3,5634.899902
1,1,p1,c1,234.899994
2,2,p2,c4,2334.899902
3,4,p4,c1,6634.899902
4,3,p3,c2,534.900024


In [21]:
session.execute(''' create table kk.orders (id int primary key ,quantity int ,prod_id int)''')

In [22]:
session.execute('''insert into kk.orders (id,quantity,prod_id) values (1,3,2)''')
session.execute('''insert into kk.orders (id,quantity,prod_id) values (2,5,1)''')
session.execute('''insert into kk.orders (id,quantity,prod_id) values (3,3,3)''')
session.execute('''insert into kk.orders (id,quantity,prod_id) values (4,5,4)''')

In [24]:
session.execute('''create table kk.users (id int primary key,name text)''')

In [25]:
session.execute('''insert into kk.users (id,name) values (1,'ali')''')
session.execute('''insert into kk.users (id,name) values (2,'ahmed')''')
session.execute('''insert into kk.users (id,name) values (3,'sara')''')

In [26]:
session.execute('''create table kk.order_details (id int primary key,user_id int,ord_id int)''')

In [27]:
session.execute('''insert into kk.order_details (id,user_id,ord_id) values (1,1,2)''')
session.execute('''insert into kk.order_details (id,user_id,ord_id) values (2,2,1)''')
session.execute('''insert into kk.order_details (id,user_id,ord_id) values (3,1,3)''')
session.execute('''insert into kk.order_details (id,user_id,ord_id) values (4,2,4)''')

In [28]:
df

,id,name,category,price
0,5,p5,c3,5634.899902
1,1,p1,c1,234.899994
2,2,p2,c1,2334.899902
3,4,p4,c1,6634.899902
4,6,p6,c2,453.700012
5,3,p3,c2,534.900024


In [30]:
df.groupby('category').id.count()

category
c1    3
c2    2
c3    1
Name: id, dtype: int64

In [32]:
df.sort_values('price',ascending=False)

,id,name,category,price
3,4,p4,c1,6634.899902
0,5,p5,c3,5634.899902
2,2,p2,c1,2334.899902
5,3,p3,c2,534.900024
4,6,p6,c2,453.700012
1,1,p1,c1,234.899994


In [33]:
result=session.execute('''select * from kk.orders''')

In [34]:
data={'id':[],'quantity':[],'prod_id':[]}

In [35]:
for r in result:
    data['id'].append(r.id)
    data['quantity'].append(r.quantity)
    data['prod_id'].append(r.prod_id)
    
df=pd.DataFrame(data)
df

,id,quantity,prod_id
0,1,3,2
1,2,5,1
2,4,5,4
3,3,3,3


In [36]:
df['quantity'].max()

5

In [41]:
max_selled_id=list(df[df['quantity']==df['quantity'].max()].prod_id.values)

In [50]:
df_prod[df_prod.id.isin(max_selled_id)].price

1     234.899994
3    6634.899902
Name: price, dtype: float64